# The process of find the best ML model

This notebook is part of **Story 2.1: ML Model Training and Persistence**.

## Data Loading

In [77]:
import sqlite3
import pandas as pd
import os
from pandas.io.sql import DatabaseError

# Path to database
db_path = '../../football.db'

if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database '{db_path}' not found. Run db_setup.py first.")

conn = sqlite3.connect(db_path)

try:
    df_raw = pd.read_sql_query("SELECT * FROM matches", conn)
    print(f"✅ Loaded {len(df_raw)} rows from 'matches' table")
except DatabaseError as e:
    df_raw = pd.DataFrame()
    print(f"❌ Error loading data: {e}")
finally:
    conn.close()

✅ Loaded 1782 rows from 'matches' table


## Feature engineering 1: add column 'Season' (for time series training-testing)

In [78]:
def add_season(df):
    df['Date'] = pd.to_datetime(df['Date'])
    def get_season(date):
        if pd.Timestamp('2024-07-26') <= date <= pd.Timestamp('2025-05-29'):
            return '2024/2025'
        elif pd.Timestamp('2023-07-28') <= date <= pd.Timestamp('2024-06-02'):
            return '2023/2024'
        elif pd.Timestamp('2022-07-22') <= date <= pd.Timestamp('2023-04-23'):
            return '2022/2023'
        elif pd.Timestamp('2021-07-23') <= date <= pd.Timestamp('2022-04-10'):
            return '2021/2022'
        elif pd.Timestamp('2020-08-08') <= date <= pd.Timestamp('2021-04-18'):
            return '2020/2021'
        elif pd.Timestamp('2019-07-26') <= date <= pd.Timestamp('2020-03-07'):
            return '2019/2020'
        else:
            return '2025/2026'

    df['Season'] = df['Date'].apply(get_season)
    return df

## Feature engineering 2: add rolling goal-related overall performance columns(features)

In [79]:
import numpy as np

def add_rolling_perf(df, rolling_window):
    # Copy and sort dataset chronologically
    df_raw = df.copy()
    df_raw = df_raw.sort_values('Date').reset_index(drop=True)
    df_raw['rowid'] = df_raw.index  # Unique ID to merge later

    # Home team features
    home_df = df_raw[['rowid', 'Date', 'HomeTeam', 'FTR', 'HS', 'HST', 'HC', 'HF', 'HY', 'HR', 'FTHG', 'FTAG']].copy()
    home_df['team'] = home_df['HomeTeam']
    home_df['side'] = 'home'
    home_df['points'] = home_df['FTR'].map({'H': 3, 'D': 1, 'A': 0})
    home_df = home_df.rename(columns={
        'HS': 'shots', 'HST': 'shots_on_target',
        'HC': 'corners', 'HF': 'fouls',
        'HY': 'yellow_cards', 'HR': 'red_cards',
        'FTHG': 'goals_scored', 'FTAG': 'goals_conceded'
    })

    # Away team features
    away_df = df_raw[['rowid', 'Date', 'AwayTeam', 'FTR', 'AS', 'AST', 'AC', 'AF', 'AY', 'AR', 'FTAG', 'FTHG']].copy()
    away_df['team'] = away_df['AwayTeam']
    away_df['side'] = 'away'
    away_df['points'] = away_df['FTR'].map({'A': 3, 'D': 1, 'H': 0})
    away_df = away_df.rename(columns={
        'AS': 'shots', 'AST': 'shots_on_target',
        'AC': 'corners', 'AF': 'fouls',
        'AY': 'yellow_cards', 'AR': 'red_cards',
        'FTAG': 'goals_scored', 'FTHG': 'goals_conceded'
    })

    # Combine and sort
    team_games = pd.concat([
        home_df[['rowid', 'Date', 'team', 'side', 'points', 'shots', 'shots_on_target',
                 'corners', 'fouls', 'yellow_cards', 'red_cards', 'goals_scored', 'goals_conceded']],
        away_df[['rowid', 'Date', 'team', 'side', 'points', 'shots', 'shots_on_target',
                 'corners', 'fouls', 'yellow_cards', 'red_cards', 'goals_scored', 'goals_conceded']]
    ])
    team_games = team_games.sort_values(by=['team', 'Date'])

    # Weight setup
    weights = np.arange(1, rolling_window + 1)
    weights = weights / weights.sum()

    def weighted_avg(series, weights_array):
        w = weights_array[-len(series):]
        return np.dot(series, w)

    # Form Score (weighted Points)
    team_games['form_score'] = (
        team_games.groupby('team')['points']
        .apply(lambda x: x.shift(1).rolling(window=rolling_window, min_periods=rolling_window)
               .apply(lambda y: weighted_avg(y, weights), raw=True))
        .reset_index(level=0, drop=True)
    )

    # Other stats (weighted Averages)
    for col in ['shots', 'shots_on_target', 'corners', 'fouls',
                'yellow_cards', 'red_cards', 'goals_scored', 'goals_conceded']:
        team_games[f'avg_{col}'] = (
            team_games.groupby('team')[col]
            .apply(lambda x: x.shift(1).rolling(window=rolling_window, min_periods=rolling_window)
                   .apply(lambda y: weighted_avg(y, weights), raw=True))
            .reset_index(level=0, drop=True)
        )

    # Split home/away features
    home_features = team_games[team_games['side'] == 'home'].copy()
    away_features = team_games[team_games['side'] == 'away'].copy()

    # Merge back to original data
    df_enriched = df_raw.merge(home_features[[
        'rowid', 'form_score', 'avg_shots', 'avg_shots_on_target', 'avg_corners',
        'avg_fouls', 'avg_yellow_cards', 'avg_red_cards',
        'avg_goals_scored', 'avg_goals_conceded'
    ]], on='rowid', how='left').rename(columns={
        'form_score': 'HomeTeam_FormScore',
        'avg_shots': 'HomeTeam_AvgShots',
        'avg_shots_on_target': 'HomeTeam_AvgShotsOnTarget',
        'avg_corners': 'HomeTeam_AvgCorners',
        'avg_fouls': 'HomeTeam_AvgFouls',
        'avg_yellow_cards': 'HomeTeam_AvgYellowCards',
        'avg_red_cards': 'HomeTeam_AvgRedCards',
        'avg_goals_scored': 'HomeTeam_AvgGoalsScored',
        'avg_goals_conceded': 'HomeTeam_AvgGoalsConceded'
    })

    df_enriched = df_enriched.merge(away_features[[
        'rowid', 'form_score', 'avg_shots', 'avg_shots_on_target', 'avg_corners',
        'avg_fouls', 'avg_yellow_cards', 'avg_red_cards',
        'avg_goals_scored', 'avg_goals_conceded'
    ]], on='rowid', how='left').rename(columns={
        'form_score': 'AwayTeam_FormScore',
        'avg_shots': 'AwayTeam_AvgShots',
        'avg_shots_on_target': 'AwayTeam_AvgShotsOnTarget',
        'avg_corners': 'AwayTeam_AvgCorners',
        'avg_fouls': 'AwayTeam_AvgFouls',
        'avg_yellow_cards': 'AwayTeam_AvgYellowCards',
        'avg_red_cards': 'AwayTeam_AvgRedCards',
        'avg_goals_scored': 'AwayTeam_AvgGoalsScored',
        'avg_goals_conceded': 'AwayTeam_AvgGoalsConceded'
    })

    return df_enriched


## Feature engineering 3: add rolling goal-related head-to-head performance columns(features)

In [80]:
import pandas as pd
import numpy as np

def add_h2h_rolling_perf(df, rolling_window):
    df_raw = df.copy().sort_values('Date').reset_index(drop=True)
    df_raw['rowid'] = df_raw.index

    # Prepare match-level team/opponent data
    home_df = df_raw[['rowid', 'Date', 'HomeTeam', 'AwayTeam', 'FTR',
                      'HS', 'HST', 'HC', 'HF', 'HY', 'HR', 'FTHG', 'FTAG']].copy()
    home_df['team'] = home_df['HomeTeam']
    home_df['opponent'] = home_df['AwayTeam']
    home_df['points'] = home_df['FTR'].map({'H': 3, 'D': 1, 'A': 0})
    home_df = home_df.rename(columns={
        'HS': 'shots', 'HST': 'shots_on_target',
        'HC': 'corners', 'HF': 'fouls',
        'HY': 'yellow_cards', 'HR': 'red_cards',
        'FTHG': 'goals_scored', 'FTAG': 'goals_conceded'
    })

    away_df = df_raw[['rowid', 'Date', 'HomeTeam', 'AwayTeam', 'FTR',
                      'AS', 'AST', 'AC', 'AF', 'AY', 'AR', 'FTAG', 'FTHG']].copy()
    away_df['team'] = away_df['AwayTeam']
    away_df['opponent'] = away_df['HomeTeam']
    away_df['points'] = away_df['FTR'].map({'A': 3, 'D': 1, 'H': 0})
    away_df = away_df.rename(columns={
        'AS': 'shots', 'AST': 'shots_on_target',
        'AC': 'corners', 'AF': 'fouls',
        'AY': 'yellow_cards', 'AR': 'red_cards',
        'FTAG': 'goals_scored', 'FTHG': 'goals_conceded'
    })

    # Combine
    matches = pd.concat([home_df, away_df], ignore_index=True)
    matches = matches.sort_values(['team', 'opponent', 'Date'])

    # Rolling weights
    weights = np.arange(1, rolling_window + 1) / np.arange(1, rolling_window + 1).sum()

    def weighted_avg(series):
        w = weights[-len(series):]
        return np.dot(series, w)

    # Compute H2H rolling stats for each (team, opponent) pair
    stat_cols = ['points', 'shots', 'shots_on_target', 'corners',
                 'fouls', 'yellow_cards', 'red_cards', 'goals_scored', 'goals_conceded']

    for col in stat_cols:
        matches[f'h2h_avg_{col}'] = (
            matches.groupby(['team', 'opponent'])[col]
            .apply(lambda x: x.shift(1).rolling(window=rolling_window, min_periods=rolling_window)
                   .apply(weighted_avg, raw=True))
            .reset_index(level=[0, 1], drop=True)
        )

    # Split back into home/away sets for merging
    home_h2h = matches[matches['team'] == matches['HomeTeam']][
        ['rowid'] + [f'h2h_avg_{c}' for c in stat_cols]
    ]
    away_h2h = matches[matches['team'] == matches['AwayTeam']][
        ['rowid'] + [f'h2h_avg_{c}' for c in stat_cols]
    ]

    # Merge back into original df
    df_enriched = df_raw.merge(
        home_h2h, on='rowid', how='left'
    ).rename(columns=lambda x: x.replace('h2h_avg_', 'HomeTeam_H2H_Avg'))

    df_enriched = df_enriched.merge(
        away_h2h, on='rowid', how='left'
    ).rename(columns=lambda x: x.replace('h2h_avg_', 'AwayTeam_H2H_Avg'))

    return df_enriched

In [63]:
df = add_season(df_raw)
df = add_rolling_perf(df, rolling_window=10)
df = add_h2h_rolling_perf(df, rolling_window=3)
df = df.iloc[:, -36:]
df = df.dropna()
list = df.columns.to_list()
print(list)

['HomeTeam_FormScore', 'HomeTeam_AvgShots', 'HomeTeam_AvgShotsOnTarget', 'HomeTeam_AvgCorners', 'HomeTeam_AvgFouls', 'HomeTeam_AvgYellowCards', 'HomeTeam_AvgRedCards', 'HomeTeam_AvgGoalsScored', 'HomeTeam_AvgGoalsConceded', 'AwayTeam_FormScore', 'AwayTeam_AvgShots', 'AwayTeam_AvgShotsOnTarget', 'AwayTeam_AvgCorners', 'AwayTeam_AvgFouls', 'AwayTeam_AvgYellowCards', 'AwayTeam_AvgRedCards', 'AwayTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsConceded', 'HomeTeam_H2H_Avgpoints', 'HomeTeam_H2H_Avgshots', 'HomeTeam_H2H_Avgshots_on_target', 'HomeTeam_H2H_Avgcorners', 'HomeTeam_H2H_Avgfouls', 'HomeTeam_H2H_Avgyellow_cards', 'HomeTeam_H2H_Avgred_cards', 'HomeTeam_H2H_Avggoals_scored', 'HomeTeam_H2H_Avggoals_conceded', 'AwayTeam_H2H_Avgpoints', 'AwayTeam_H2H_Avgshots', 'AwayTeam_H2H_Avgshots_on_target', 'AwayTeam_H2H_Avgcorners', 'AwayTeam_H2H_Avgfouls', 'AwayTeam_H2H_Avgyellow_cards', 'AwayTeam_H2H_Avgred_cards', 'AwayTeam_H2H_Avggoals_scored', 'AwayTeam_H2H_Avggoals_conceded']


## Prepare train and test sets: ##
*Note: Apply mannual preprocessing (i.e., feature engineering1 + feature engineering2) to dataset before feeding into sklearn/H2o/NN pipelines.*

In [81]:
def prepare_train_test_sets(df_raw, rolling_window_overall, rolling_window_h2h, selected_features, test_seasons, train_seasons):
    # Add 'Season' and rolling performance cols to dataset, drop null values
    df = add_season(df_raw)
    df = add_rolling_perf(df, rolling_window=rolling_window_overall)
    df = add_h2h_rolling_perf(df, rolling_window=rolling_window_h2h)
    df = df[selected_features + ['FTR'] +['Season']].dropna()

    # Prepare train and test subsets
    train_df = df[df['Season'].isin(train_seasons)]
    train_df = train_df[selected_features + ['FTR']]
    test_df = df[df['Season'].isin(test_seasons)]
    test_df = test_df[selected_features + ['FTR']]
    X_train = train_df[selected_features]
    y_train = train_df['FTR']
    X_test = test_df[selected_features]
    y_test = test_df['FTR']

    return train_df, test_df, X_train, y_train, X_test, y_test

## Choose the best single conventional ML model in sklearn (Metric: acc)
*Loop all combinations of features & model*

*Settings: train on 2019-2024 seasons, test on 2024-2025 season.*

### Try process

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from scipy.special import softmax
import itertools

# Loop combination of features and model to find the best model:
feature_groups = [['B365H', 'B365D', 'B365A'], # odds, mannually chosed
                     ['HomeTeam_FormScore', 'AwayTeam_FormScore'],
                     ['HomeTeam_AvgShots', 'AwayTeam_AvgShots'],
                     ['HomeTeam_AvgShotsOnTarget', 'AwayTeam_AvgShotsOnTarget'],
                     ['HomeTeam_AvgCorners', 'AwayTeam_AvgCorners'],
                     ['HomeTeam_AvgFouls', 'AwayTeam_AvgFouls'],
                     ['HomeTeam_AvgYellowCards', 'AwayTeam_AvgYellowCards'],
                     ['HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards'],
                     ['HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored'],
                     ['HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded']]

metric_ultra_max = 0
model_ultra_max = "Init"
features_ultra_max = []

for r in range(1, len(feature_groups) + 1):
    for combo in itertools.combinations(feature_groups, r):
        # Prepare selected features
        selected_features = ['HomeTeam', 'AwayTeam'] + [feature for group in combo for feature in group]
        print(f"📂 Selected features: {selected_features}")

        # Prepare train and test subsets
        _, _, X_train, y_train, X_test, y_test = prepare_train_test_sets(df_raw, rolling_window=10, selected_features=selected_features, test_seasons=['2024/2025'], train_seasons=['2019/2020', '2020/2021', '2021/2022','2022/2023','2023/2024'])

        # Make clear cat and num features in X for later sklearn pipeline
        categorical_features = ['HomeTeam', 'AwayTeam']
        numerical_features = [col for col in X_train.columns if col not in categorical_features]

        # Prepare sklearn pipelines.
        # Note that most models support one-hot, for models do not supporting sparse matrix (HistGradientBoosting, NaiveBayes, QDA), use ordinal encoding instead
        # Note that imputer is useless if previous already dropna. Scaler is optional, we just applied it here.
        numeric_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        # optional: PCA (proven to be useless, discard :( )
        # numeric_pipeline = numeric_pipeline = Pipeline([
        #     ('imputer', SimpleImputer(strategy='mean')),
        #     ('scaler', StandardScaler()),
        #     ('pca', PCA(n_components=0.95))
        # ])
        categorical_pipeline1 = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])
        categorical_pipeline2 = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ])
        preprocessor1 = ColumnTransformer([
            ('num', numeric_pipeline, numerical_features),
            ('cat', categorical_pipeline1, categorical_features)
        ])
        preprocessor2 = ColumnTransformer([
            ('num', numeric_pipeline, numerical_features),
            ('cat', categorical_pipeline2, categorical_features)
        ])
        models_and_preprocessors = {
            "RandomForest": [RandomForestClassifier(random_state=42), preprocessor1],
            "LogisticRegression": [LogisticRegression(max_iter=1000, random_state=42), preprocessor1],
            "PassiveAggressiveClassifier": [PassiveAggressiveClassifier(max_iter=1000, random_state=42), preprocessor1],
            "RidgeClassifier": [RidgeClassifier(max_iter=1000), preprocessor1],
            "KNN": [KNeighborsClassifier(), preprocessor1],
            "SVC": [SVC(random_state=42), preprocessor1],
            "DecisionTree": [DecisionTreeClassifier(random_state=42), preprocessor1],
            "GradientBoosting": [GradientBoostingClassifier(random_state=42), preprocessor1],
            "HistGradientBoosting": [HistGradientBoostingClassifier(random_state=42), preprocessor2],
            "AdaBoost": [AdaBoostClassifier(random_state=42), preprocessor1],
            "ExtraTrees": [ExtraTreesClassifier(random_state=42), preprocessor1],
            "NaiveBayes": [GaussianNB(), preprocessor2],
            "MLP": [MLPClassifier(max_iter=1000, random_state=42), preprocessor1],
            "QDA": [QuadraticDiscriminantAnalysis(), preprocessor2],
            "LDA": [LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'), preprocessor2]
        }

        # Loop to train in different models and display selected features and accuracy
        acc_max = 0
        model_max = "Init"
        for name, model_and_proprocessor in models_and_preprocessors.items():
            model_pipeline = Pipeline([
                ('preprocess', model_and_proprocessor[1]),
                ('classifier', model_and_proprocessor[0])
            ])
            model_pipeline.fit(X_train, y_train)

            y_pred = model_pipeline.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            print(f"🎯 {name} Accuracy: {acc:.4f}")
            if acc > acc_max:
                acc_max = acc
                model_max = name

        
        # Update the best model in this selected features
        if acc_max > metric_ultra_max:
            metric_ultra_max = acc_max
            model_ultra_max = model_max
            features_ultra_max = selected_features
        
    print(f"The best model until now is {model_ultra_max}.\nThe accuracy is {metric_ultra_max}.\nUsed features: {features_ultra_max}.")

**Result:**

The best model is **LDA** with **05404** accuracy using the feature combination: ['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'HomeTeam_FormScore', 'AwayTeam_FormScore', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards', 'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored'].

The best model without betting features is **RidgeClassifier** with **0.5292** accuracy using the feature combination: ['HomeTeam', 'AwayTeam', 'HomeTeam_FormScore', 'AwayTeam_FormScore', 'HomeTeam_AvgShots', 'AwayTeam_AvgShots', 'HomeTeam_AvgFouls', 'AwayTeam_AvgFouls', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards']

See **sklearn_model_acc.txt** the records of all models' accuracy.

### Is there a way to improve the chosed model (tuning hyperparams?)

#### 1. Check and tune LDA

##### 1.1 Check LDA

In [ ]:
from pipeline import pipeline_LDA
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Check the draw of LDA
print("😀Train LDA on 2019-2024, test on 2024-2025, check the test probabilities of Draw")
selected_features = ['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'HomeTeam_FormScore', 'AwayTeam_FormScore', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards', 'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored']
_, _, X_train, y_train, X_test, y_test = prepare_train_test_sets(df_raw, rolling_window=10, selected_features=selected_features, test_seasons=['2024/2025'], train_seasons=['2019/2020', '2020/2021', '2021/2022','2022/2023','2023/2024'])
pipe_LDA = pipeline_LDA()
pipe_LDA.fit(X_train, y_train)

probs = pipe_LDA.predict_proba(X_test)
label_to_index = {label: idx for idx, label in enumerate(pipe_LDA.classes_)}
probs_H = probs[:, label_to_index['H']]
probs_D = probs[:, label_to_index['D']]
probs_A = probs[:, label_to_index['A']]
results_df = X_test.copy()
results_df['Prob_H'] = probs_H
results_df['Prob_D'] = probs_D
results_df['Prob_A'] = probs_A
display_cols = ['HomeTeam', 'AwayTeam', 'Prob_H', 'Prob_D', 'Prob_A']
print(results_df[display_cols].to_string(index=False, float_format='%.3f'))

y_pred = pipe_LDA.predict(X_test)
print(classification_report(y_test, y_pred, target_names=pipe_LDA.classes_))

acc = accuracy_score(y_test, y_pred)
print(f"🎯 Accuracy: {acc:.4f}")

**Result**: the draw is only predicted for 3 times, although the acc of predicted is high. Let's add Parity features and threshold tuning to solve this problem.

*Class 'A' (Away Win): Precision: 0.44 → When the model predicts "A", it's right 44% of the time. Recall: 0.65 → Out of all actual "A" games, it correctly identified 65%.*

*Class 'D' (Draw): Precision: 1.00 → The model only predicted "D" when it was correct — but this is misleading. Recall: 0.05 → It caught only 5% of actual draws! Terrible sensitivity.*

*Class 'H' (Home Win): Precision: 0.59 → Not bad, 59% correct when predicting "H". Recall: 0.79 → It caught 79% of all actual home wins.*

##### 1.2 Tune with parity features

In [ ]:
import itertools
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

selected_features = ['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'HomeTeam_FormScore', 'AwayTeam_FormScore', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards', 'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored']
_, _, X_train, y_train, X_test, y_test = prepare_train_test_sets(df_raw, rolling_window=10, selected_features=selected_features, test_seasons=['2024/2025'], train_seasons=['2019/2020', '2020/2021', '2021/2022','2022/2023','2023/2024'])
X_train['FormDiff'] = abs(X_train['HomeTeam_FormScore'] - X_train['AwayTeam_FormScore'])
X_test['FormDiff'] = abs(X_test['HomeTeam_FormScore'] - X_test['AwayTeam_FormScore'])
X_train['RedcardsDiff'] = abs(X_train['HomeTeam_AvgRedCards'] - X_train['AwayTeam_AvgRedCards'])
X_test['RedcardsDiff'] = abs(X_test['HomeTeam_AvgRedCards'] - X_test['AwayTeam_AvgRedCards'])
X_train['GoalsscoredDiff'] = abs(X_train['HomeTeam_AvgGoalsScored'] - X_train['AwayTeam_AvgGoalsScored'])
X_test['GoalsscoredDiff'] = abs(X_test['HomeTeam_AvgGoalsScored'] - X_test['AwayTeam_AvgGoalsScored'])

added_feature_groups = [['FormDiff'], ['RedcardsDiff'],['GoalsscoredDiff']]

for r in range(1, len(added_feature_groups) + 1):
    for combo in itertools.combinations(added_feature_groups, r):
        numerical_features = ['B365H', 'B365D', 'B365A', 'HomeTeam_FormScore', 'AwayTeam_FormScore', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards', 'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored'] + [feature for group in combo for feature in group]
        print(f"😀 numerical_features: {numerical_features}")
        pipe_LDA = Pipeline([
            ('ColumnTransform', ColumnTransformer([
                ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())]), numerical_features),
                ('cat', Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')), ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))]), ['HomeTeam', 'AwayTeam'])
            ])),
            ('classifier', LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'))
        ])
        pipe_LDA.fit(X_train, y_train)
        probs = pipe_LDA.predict_proba(X_test)
        labels = pipe_LDA.classes_
        label_to_index = {label: i for i, label in enumerate(labels)}
        probs_H = probs[:, label_to_index['H']]
        probs_D = probs[:, label_to_index['D']]
        probs_A = probs[:, label_to_index['A']]
        results_df = X_test.copy()
        results_df['Prob_H'] = probs_H
        results_df['Prob_D'] = probs_D
        results_df['Prob_A'] = probs_A
        display_cols = ['HomeTeam', 'AwayTeam', 'Prob_H', 'Prob_D', 'Prob_A']
        print(results_df[display_cols].to_string(index=False, float_format='%.3f'))
        y_pred = pipe_LDA.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"🎯 Accuracy: {acc:.4f}")
        print(classification_report(y_test, y_pred, target_names=labels))

**Result**: not big improvements, let's try threshold tuning!

##### 1.3 Tune with threshold tuning

In [ ]:
from pipeline import pipeline_LDA
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

selected_features = ['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'HomeTeam_FormScore', 'AwayTeam_FormScore', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards', 'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored']
_, _, X_train, y_train, X_test, y_test = prepare_train_test_sets(df_raw, rolling_window=10, selected_features=selected_features, test_seasons=['2024/2025'], train_seasons=['2019/2020', '2020/2021', '2021/2022','2022/2023','2023/2024'])
pipe_LDA = pipeline_LDA()
pipe_LDA.fit(X_train, y_train)

probs = pipe_LDA.predict_proba(X_test)
labels = pipe_LDA.classes_
label_to_index = {label: idx for idx, label in enumerate(labels)}

delta = 0.0000001
y_pred = []
for p in probs:
    prob_H = p[label_to_index['H']]
    prob_A = p[label_to_index['A']]
    if abs(prob_H - prob_A) < delta:
        y_pred.append('D')
    else:
        y_pred.append(labels[np.argmax(p)])

# Step 5: Build results DataFrame
results_df = X_test[['HomeTeam', 'AwayTeam']].copy()
results_df['Prob_H'] = probs[:, label_to_index['H']]
results_df['Prob_D'] = probs[:, label_to_index['D']]
results_df['Prob_A'] = probs[:, label_to_index['A']]
results_df['Predicted'] = y_pred
results_df['Actual'] = y_test.values

# Step 6: Print results and metrics
print(results_df[['HomeTeam', 'AwayTeam', 'Prob_H', 'Prob_D', 'Prob_A', 'Predicted', 'Actual']].to_string(index=False, float_format='%.3f'))

acc = accuracy_score(y_test, y_pred)
print(f"\n🎯 Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=labels))

**Result**: not big improvements, give up!

##### Bonus: try this model in broer score

In [ ]:
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import LabelBinarizer

print(y_test)
lb = LabelBinarizer()
y_true_bin = lb.fit_transform(y_test) # One-hot encoding for ['A', 'D', 'H']
print(y_true_bin)

# Match predicted probs to label order from binarizer
labels = pipe_LDA.classes_
label_to_index = {label: idx for idx, label in enumerate(labels)}
probs_ordered = np.zeros_like(y_true_bin, dtype=float)
for idx, label in enumerate(labels):
    probs_ordered[:, idx] = probs[:, label_to_index[label]]
print(probs_ordered)

brier_multi = np.mean((probs_ordered - y_true_bin) ** 2)
print(f"📌 Overall Multi-class Brier Score: {brier_multi:.4f}")

# Per-Class Brier Scores
brier_per_class = {}
for label in labels:  # ['A', 'D', 'H']
    y_true_binary = (y_test == label).astype(int)
    y_prob = probs[:, label_to_index[label]]
    brier_per_class[label] = brier_score_loss(y_true_binary, y_prob)

print("📌 Per-class Brier Scores:")
for label, score in brier_per_class.items():
    print(f"{label}: {score:.4f}")

## Choose the best single conventional ML model in sklearn (Metric: brier score)
*Loop all combinations of features & model*

*Settings: train on 2019-2024 seasons, test on 2024-2025 season.*

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
# from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
# from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import brier_score_loss
# from scipy.special import softmax
# from sklearn.preprocessing import LabelBinarizer
import itertools

from sklearn.model_selection import TimeSeriesSplit
from sklearn.calibration import CalibratedClassifierCV

# Loop combination of features and model to find the best model:
feature_groups = [#['B365H', 'B365D', 'B365A'], # odds, mannually chosed
                     ['HomeTeam_FormScore', 'AwayTeam_FormScore'],
                     ['HomeTeam_AvgShots', 'AwayTeam_AvgShots'],
                     ['HomeTeam_AvgShotsOnTarget', 'AwayTeam_AvgShotsOnTarget'],
                     ['HomeTeam_AvgCorners', 'AwayTeam_AvgCorners'],
                     ['HomeTeam_AvgFouls', 'AwayTeam_AvgFouls'],
                     ['HomeTeam_AvgYellowCards', 'AwayTeam_AvgYellowCards'],
                     ['HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards'],
                     ['HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored'],
                     ['HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded'],
                     ['HomeTeam_H2H_Avgpoints', 'AwayTeam_H2H_Avgpoints'],
                     ['HomeTeam_H2H_Avgshots', 'AwayTeam_H2H_Avgshots'],
                     ['HomeTeam_H2H_Avgshots_on_target', 'AwayTeam_H2H_Avgshots_on_target'],
                     ['HomeTeam_H2H_Avgcorners', 'AwayTeam_H2H_Avgcorners'],
                     ['HomeTeam_H2H_Avgfouls', 'AwayTeam_H2H_Avgfouls'],
                     ['HomeTeam_H2H_Avgyellow_cards', 'AwayTeam_H2H_Avgyellow_cards'],
                     ['HomeTeam_H2H_Avgred_cards', 'AwayTeam_H2H_Avgred_cards'],
                     ['HomeTeam_H2H_Avggoals_scored', 'AwayTeam_H2H_Avggoals_scored'],
                     ['HomeTeam_H2H_Avggoals_conceded', 'AwayTeam_H2H_Avggoals_conceded']]

brier_ultra_best = 1 # Initialize it with the worst, i.e., 1
model_ultra_best = "Init"
features_ultra_best = []

for r in range(1, len(feature_groups) + 1):
    for combo in itertools.combinations(feature_groups, r):
        # Prepare selected features
        selected_features = [feature for group in combo for feature in group] + ['HomeTeam', 'AwayTeam'] 
        print(f"📂 Selected features: {selected_features}")

        # Prepare train and test subsets
        _, _, X_train, y_train, X_test, y_test = prepare_train_test_sets(df_raw, rolling_window_overall=8, rolling_window_h2h=3, selected_features=selected_features, test_seasons=['2024/2025'], train_seasons=['2019/2020', '2020/2021', '2021/2022','2022/2023','2023/2024'])

        # Make clear cat and num features in X for later sklearn pipeline
        categorical_features = ['HomeTeam', 'AwayTeam']
        numerical_features = [col for col in X_train.columns if col not in categorical_features]

        # Prepare sklearn pipelines.
        # Note that most models support one-hot, for models do not supporting sparse matrix (HistGradientBoosting, NaiveBayes, QDA), use ordinal encoding instead
        # Note that imputer is useless if previous already dropna. Scaler is optional, we just applied it here.
        numeric_pipeline = Pipeline([
            #('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        categorical_pipeline1 = Pipeline([
            #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])
        categorical_pipeline2 = Pipeline([
            #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ])
        preprocessor1 = ColumnTransformer([
            ('num', numeric_pipeline, numerical_features),
            ('cat', categorical_pipeline1, categorical_features)
        ])
        preprocessor2 = ColumnTransformer([
            ('num', numeric_pipeline, numerical_features),
            ('cat', categorical_pipeline2, categorical_features)
        ])
        models_and_preprocessors = {
            "RandomForest": [RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_leaf=5, max_features='sqrt', random_state=42), preprocessor1],
            "LogisticRegression": [LogisticRegression(max_iter=1000, C=1.0, solver='lbfgs', multi_class='multinomial', random_state=42), preprocessor1],
            "PassiveAggressiveClassifier": [PassiveAggressiveClassifier(max_iter=1000, C=0.5, random_state=42), preprocessor1],
            "RidgeClassifier": [RidgeClassifier(max_iter=1000, alpha=1.0), preprocessor1],
            "KNN": [KNeighborsClassifier(n_neighbors=25, weights='distance'), preprocessor1],
            "SVC": [SVC(C=1.0, kernel='rbf', gamma='scale', probability=True, random_state=42), preprocessor1],
            "DecisionTree": [DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, random_state=42), preprocessor1],
            "GradientBoosting": [GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=3, min_samples_leaf=5, random_state=42), preprocessor1],
            "HistGradientBoosting": [HistGradientBoostingClassifier(learning_rate=0.05, max_depth=6, min_samples_leaf=10, max_iter=1000, random_state=42), preprocessor2],
            "AdaBoost": [AdaBoostClassifier(n_estimators=500, learning_rate=0.5, random_state=42), preprocessor1],
            "ExtraTrees": [ExtraTreesClassifier(n_estimators=500, max_depth=None, min_samples_leaf=5, max_features='sqrt', random_state=42), preprocessor1],
            "NaiveBayes": [GaussianNB(), preprocessor2],
            "MLP": [MLPClassifier(max_iter=1000, hidden_layer_sizes=(100, 50), alpha=1e-4, learning_rate_init=1e-3, random_state=42), preprocessor1],
            "QDA": [QuadraticDiscriminantAnalysis(reg_param=0.1), preprocessor2],
            "LDA": [LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'), preprocessor2]
        }

        # Loop to train in different models and display brier score
        brier_best = 1
        model_best = "Init"
        for name, model_and_proprocessor in models_and_preprocessors.items():
            model_pipeline = Pipeline([
                ('preprocess', model_and_proprocessor[1]),
                ('classifier', model_and_proprocessor[0])
            ])
            model_pipeline.fit(X_train, y_train)
            # Display brier score
            '''Not use calibration
            try:
                probs = model_pipeline.predict_proba(X_test)
            except:
                raw_scores = model_pipeline.decision_function(X_test)
                if raw_scores.ndim == 1:
                    raw_scores = np.vstack([-raw_scores, raw_scores]).T
                probs = softmax(raw_scores, axis=1)
            lb = LabelBinarizer()
            y_true_bin = lb.fit_transform(y_test) # One-hot encoding for ['A', 'D', 'H']
            labels = model_pipeline.classes_
            label_to_index = {label: idx for idx, label in enumerate(labels)}
            probs_ordered = np.zeros_like(y_true_bin, dtype=float)
            for idx, label in enumerate(labels):
                probs_ordered[:, idx] = probs[:, label_to_index[label]]
            brier = np.mean((probs_ordered - y_true_bin) ** 2)
            '''

            # Calibration for non-proba models
            if name == "NaiveBayes" or name == "QDA" or name == "LDA":
                y_proba = model_pipeline.predict_proba(X_test)
            else:
                tscv = TimeSeriesSplit(n_splits=5)
                calibrated = CalibratedClassifierCV(
                    estimator=model_pipeline,
                    method='isotonic',
                    cv=tscv
                )
                calibrated.fit(X_train, y_train)
                y_proba = calibrated.predict_proba(X_test)
            brier = brier_score_loss(y_test, y_proba)/3
            print(f"🎯 {name} brier score: {brier:.4f}")

            # Update the best brier score in this selected features
            if brier < brier_best:
                brier_best = brier
                model_best = name

        # Update the best model until now
        if  brier_best < brier_ultra_best:
            brier_ultra_best = brier_best
            model_ultra_best = model_best
            features_ultra_best = selected_features
        print(f"The best model until now is {model_ultra_best} with brier score {brier_ultra_best}.")

📂 Selected features: ['HomeTeam_FormScore', 'AwayTeam_FormScore']


ValueError: A given column is not a column of the dataframe

**Result:**

1. With simple hyperparams

    The best model is **LDA** with **0.59358586047** brier score using the feature combination: ['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'HomeTeam_AvgFouls', 'AwayTeam_AvgFouls', 'HomeTeam_AvgYellowCards', 'AwayTeam_AvgYellowCards', 'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards'].

    The best model without betting features is **RidgeClassifier** with **0.6111** brier score using the feature combination: ['HomeTeam', 'AwayTeam', 'HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded']

    See **sklearn_model_brier.txt** the records of all models' brier score.

2. With more hyperparams

    The best model without betting features is **ExtraTrees** with **0.60818908679** brier score using the feature combination: ['HomeTeam', 'AwayTeam', 'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored', 'HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded']

    See **sklearn_model_brier_morehyperparams.txt** the records of all models' brier score.

3. With calibration for some models that does not have proba

    The best model without betting features is **？** with **？** brier score using the feature combination: ['HomeTeam', 'AwayTeam', ]

    See **sklearn_model_brier_calibration.txt** the records of all models' brier score.

## Choose the best mixed model in H2O AutoML
*H2O automately selects features and decide mixed models*

*Settings: train on 2019-2024 seasons, test on 2024-2025 season.*

In [ ]:
import h2o
from h2o.automl import H2OAutoML
from h2o.automl import get_leaderboard
from sklearn.metrics import accuracy_score

# Initiate h2o with the train and test set
h2o.init()
train_df, test_df, _, _, _, _ = prepare_train_test_sets(df_raw, rolling_window=10, selected_features="all", test_seasons=['2024/2025'], train_seasons=['2019/2020', '2020/2021', '2021/2022','2022/2023','2023/2024'])
train_h2o = h2o.H2OFrame(train_df)
test_h2o = h2o.H2OFrame(test_df)

# Ensure H2o know the target is a classification problem
target_col = 'FTR'
train_h2o[target_col] = train_h2o[target_col].asfactor()
test_h2o[target_col] = test_h2o[target_col].asfactor()

# Ensure H2o know the categorial cols in predictor features is categorial
for col in ['HomeTeam', 'AwayTeam']:
    train_h2o[col] = train_h2o[col].asfactor()
    test_h2o[col] = test_h2o[col].asfactor()

# Set the param for automl and train it
aml = H2OAutoML(
    # max_models=50, # try maximum 50 models
    max_runtime_secs=600, # limit total runtime to 600 seconds
    balance_classes=True, # upsample the minority classes - "Draw"
    sort_metric='logloss', # good for multi-class classification
    nfolds=5, # use 5-fold cross-validation
    stopping_metric='logloss', # early stopping based on log loss
    stopping_rounds=30, # early stop after 30 rounds of no improvement
    seed=42 # random state 42
)
aml.train(x=[col for col in train_h2o.columns if col != target_col], y=target_col, training_frame=train_h2o)

# Get the best mixed model from the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')
top_models = lb.head(rows=1) # only need the top 1 model here
model_id = top_models.as_data_frame().iloc[0]['model_id']
model = h2o.get_model(model_id)

# Show model and train-val info
if model.algo == "stackedensemble":
    print(f"🖥️ {model.metalearner()}")
else:
    print(f"🖥️ {model.algo}")
    print("Info:", model._model_json['output'])
            
# Display accuracy
preds = model.predict(test_h2o).as_data_frame()['predict']
true = test_h2o[target_col].as_data_frame()[target_col]
acc = accuracy_score(true, preds)
print(f"🎯 Accuracy: {acc:.4f}")

**Result:**

The best accuracy is **0.5037**, worse than single model, not adopted.